<a href="https://colab.research.google.com/github/DavoodSZ1993/RL/blob/main/08_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gym
import math                                  # Provides access to the mathematical functions
import random                                # Implements pseudo-random number generators for various distributions
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count                 # Make an iterator that returns evenly spaced values starting with number start
from PIL import Image                       # Provides a number of factory functions including functions to load images from files

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

## `collections.namedtuple()` Factory Function for Tuples with Named Fields.
Named tuples assign meaning to each position in a tuple and allow for more readable, self-documenting code. 

```
>>> # Basic example
>>> Point = namedtuple('Point', ['x', 'y'])
>>> p = Point(11, y=22)     # instantiate with positional or keyword arguments
>>> p[0] + p[1]             # indexable like the plain tuple (11, 22)
33
>>> x, y = p                # unpack like a regular tuple
>>> x, y
(11, 22)
>>> p.x + p.y               # fields also accessible by name
33
>>> p                       # readable __repr__ with a name=value style
Point(x=11, y=22)
```

## `collections.deque()`:
Returns a new deque object inialized left-to-right (append()) with data from iterable. if iterable is not specified, the new deque is empty. 

```
>>> from collections import deque
>>> d = deque('ghi')                 # make a new deque with three items
>>> for elem in d:                   # iterate over the deque's elements
...     print(elem.upper())
G
H
I
```

In [2]:
env = gym.make('CartPole-v0').unwrapped

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
  from IPython import display      # Public API for display tools in IPython

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## `gym.make().unwrapped`:

Open AI Gym offers many different environments. Each of them with their own set of parameters and methods. Nevertheless they generally are wrapped by a single Class (like an interface on real OOPLs) called `Env.` This class exposes the common most essential methods of any environment, like `step`, `reset` and `seed`. Having this “interface” class is great, because it allows your code to be environment agnostic. It is also makes things easier if you want to test a single agent on different environments.

However, if you want to access the behind-the.scenes dynamics of a specific environment, then you use the `unwrapped` property.

## `matplotlib.pyplot.ion()`:
This method is used to turn on interactive mode. 

In [3]:
Transition = namedtuple('Transition',('state', 'action','next_state', 'reward'))

class Replaymemory(object):
  def __init__(self, capacity):
    self.memory = deque([], maxlen=capacity)

  def push(self, *args):
    """Save a Transition"""
    self.memory.append(Transition(*args))

  def sample(self, batch_size):
    return random.sample(self.memory, batch_size)

  def __len__(self):
    return len(self.memory)

In [4]:
class DQN(nn.Module):
  def __init__(self, h, w, outputs):
    super(DQN, self).__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)     # Convolution Layer
    self.bn1 = nn.BatchNorm2d(16)                              # Batch Normalization
    self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
    self.bn2 = nn.BatchNorm2d(32)
    self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
    self.bn3 = nn.BatchNorm2d(32)

    def conv2d_size_out(size, kernel_size=5, stride=2):
      return (size - (kernel_size - 1) -1) // stride +1

    convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
    convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
    linear_input_size = convw * convh * 32

    self.head = nn.Linear(linear_input_size, outputs)

  def forward(self, x):
    x = x.to(device)
    x = F.relu(self.bn1(self.conv1(x)))
    x = F.relu(self.bn2(self.conv2(x)))
    x = F.relu(self.bn3(self.conv3(x)))
    return self.head(x.view(x.size(0), -1))



In [5]:
resize = T.Compose([T.ToPILImage(), # Converts a tensor or an ndarray to PIL Image
                    T.Resize(40, interpolation=Image.CUBIC), # Resize the input image to the given size
                    T.ToTensor()])   # Converts a PIL image or numpy.ndarray to tensor

def get_cart_location(screen_width):
  world_width = env.x_threshold * 2
  scale = screen_width / world_width
  return int(env.state[0] * scale + screen_width / 2.0)

def get_screen():

  # Returned screen requested by gym is 400x600x3, but sometimes larger
  # such as 800x1200x3. Transpose it onto torch order (CHW)
  screen = env.render(mode='rgb_array').transpose((2, 0, 1))

  # Cart is in the lower half, so strip off the top and bottom of the screen
  _, screen_height, screen_width = screen.shape
  screen = screen[:, int(screen_height*0.4):int(screen_height * 0.8)]
  view_width = int(screen_width * 0.6)
  cart_location = get_cart_location(screen_width)

  if cart_location < view_width // 2:
    slice_range = slice(view_width)
  elif cart_location > (screen_width - view_width // 2):
    slice_range = slice(-view_width, None)
  else:
    slice_range = slice(cart_location - view_width // 2,
                        cart_location + view_width // 2)
    
  # Strip off the edges, so that we have a square image centered on a cart
  screen = screen[:, :, slice_range]

  # Convert to float, rescale, covert to torch tensor
  # (this doesn't require a copy)
  screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
  screen = torch.from_numpy(screen)
  
  # Resize, and add a batch dimension (BCHW)
  return resize(screen).unsqueeze(0)

env.reset()
plt.figure()
plt.imshow(get_screen().cpu().squeeze(0).permute(1, 2, 0).numpy(),
           interpolation='none')
plt.title('Example extracted screen')
plt.show()

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


NoSuchDisplayException: ignored

<Figure size 432x288 with 0 Axes>

### The Cause of Error:

The follwoing Q&A is from [stackoverflow](https://stackoverflow.com/):

**Question:**
[OpenAI Gym NameError in Google Colaboratory](https://stackoverflow.com/questions/49203023/openai-gym-nameerror-in-google-colaboratory):

**Answer:**
The Gym will normally render the display using GL, on the screen.

But Colab is run on the web as a notebook. It can't display to your screen. It can only show the result through HTML. 

If someone modifies the Gym to manipulate WebGL. Maybe someday, but not now.